## Recovery with addm_toolbox.ddm_mla_test

In [2]:
from __future__ import absolute_import

import numpy as np
import pkg_resources

from builtins import range, str
from multiprocessing import Pool

from addm_toolbox.ddm_mla import DDM
from addm_toolbox.util import load_trial_conditions_from_csv

In [8]:
d = .006
sigma = .08
rangeD = [0.005, 0.006, 0.007]
rangeSigma = [0.065, 0.08, 0.095]
trialsFileName=None
numTrials=10
numSimulations=10
binStep=100
maxRT=8000
numThreads = 9
verbose = False

In [4]:
def wrap_ddm_get_model_log_likelihood(args):
    """
    Wrapper for DDM.get_model_log_likelihood(), intended for parallel
    computation using a threadpool.
    Args:
      args: a tuple where the first item is a DDM object, and the remaining
          item are the same arguments required by
          DDM.get_model_log_likelihood().
    Returns:
      The output of DDM.get_model_log_likelihood().
    """
    model = args[0]
    return model.get_model_log_likelihood(*args[1:])

In [ ]:
pool = Pool(numThreads)

histBins = list(range(0, maxRT + binStep, binStep))

# Load trial conditions.
if not trialsFileName:
    trialsFileName = pkg_resources.resource_filename(
        u"addm_toolbox", u"test_data/test_trial_conditions.csv")
trialConditions = load_trial_conditions_from_csv(trialsFileName)

# Generate artificial data.
dataRTLeft = dict()
dataRTRight = dict()
for trialCondition in trialConditions:
    dataRTLeft[trialCondition] = list()
    dataRTRight[trialCondition] = list()
model = DDM(d, sigma)
for trialCondition in trialConditions:
    t = 0
    while t < numTrials:
        try:
            trial = model.simulate_trial(
                trialCondition[0], trialCondition[1])
        except:
            print(u"An exception occurred while generating artificial "
                  "trial " + str(t) + u" for condition " +
                  str(trialCondition[0]) + u", " + str(trialCondition[1]) +
                  u".")
            raise
        if trial.choice == -1:
            dataRTLeft[trialCondition].append(trial.RT)
        elif trial.choice == 1:
            dataRTRight[trialCondition].append(trial.RT)
        t += 1

# Generate histograms for artificial data.
dataHistLeft = dict()
dataHistRight = dict()
for trialCondition in trialConditions:
    dataHistLeft[trialCondition] = np.histogram(
        dataRTLeft[trialCondition], bins=histBins)[0]
    dataHistRight[trialCondition] = np.histogram(
        dataRTRight[trialCondition], bins=histBins)[0]

# Grid search on the parameters of the model.
if verbose:
    print(u"Performing grid search over the model parameters...")
listParams = list()
models = list()
for d in rangeD:
    for sigma in rangeSigma:
        model = DDM(d, sigma)
        models.append(model)
        listParams.append((model, trialConditions, numSimulations,
                          histBins, dataHistLeft, dataHistRight))
logLikelihoods = pool.map(wrap_ddm_get_model_log_likelihood, listParams)
pool.close()

if verbose:
    for i, model in enumerate(models):
        print(u"L" + str(model.params) + u" = " + str(logLikelihoods[i]))
    bestIndex = logLikelihoods.index(max(logLikelihoods))
    print(u"Best fit: " + str(models[bestIndex].params))

Process SpawnPoolWorker-12:
Traceback (most recent call last):
  File "/Users/zeynepenkavi/.pyenv/versions/3.8.3/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/zeynepenkavi/.pyenv/versions/3.8.3/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/zeynepenkavi/.pyenv/versions/3.8.3/lib/python3.8/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/Users/zeynepenkavi/.pyenv/versions/3.8.3/lib/python3.8/multiprocessing/queues.py", line 358, in get
    return _ForkingPickler.loads(res)
AttributeError: Can't get attribute 'wrap_ddm_get_model_log_likelihood' on <module '__main__' (built-in)>
Process SpawnPoolWorker-18:
Traceback (most recent call last):
  File "/Users/zeynepenkavi/.pyenv/versions/3.8.3/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/zeynepenkavi/.pyenv/versions/3.8.3/lib/python3.8/multiproce